# Final Project - A Complete Data Preparation Pipeline

# Classification Project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from feature_engine.encoding import OrdinalEncoder
from feature_engine.imputation import ArbitraryNumberImputer
from feature_engine.imputation import CategoricalImputer


In [2]:
# Loading the titanic dataset the problem will be to predict if the passenger survived when the titanic sank

titanic_data = sns.load_dataset("titanic")

titanic_data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
#Filter the dataset to only 8 columns

cols = ["pclass","sex","age","sibsp","parch","fare","embarked","survived"]

titanic_data = titanic_data[cols]

In [4]:
titanic_data.head()

,pclass,sex,age,sibsp,parch,fare,embarked,survived
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [5]:
#Viewing the different data types for each column

titanic_data.dtypes

pclass        int64
sex          object
age         float64
sibsp         int64
parch         int64
fare        float64
embarked     object
survived      int64
dtype: object

In [6]:
#Finding the ratio of missing values in all the columns

titanic_data.isnull().mean()

pclass      0.000000
sex         0.000000
age         0.198653
sibsp       0.000000
parch       0.000000
fare        0.000000
embarked    0.002245
survived    0.000000
dtype: float64

In [7]:
#Looks like 19% of the age columns data is missing and 0.002 of the data is missing in the embarked column

#Splitting the data into training and test sets


X_train,X_test,y_train,y_test = train_test_split(titanic_data.drop("survived",axis=1),titanic_data["survived"],test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((712, 7), (179, 7))

In [8]:
#Creating the Pipeline for feature engineering

titanic_data_pipe = Pipeline([
    ("numerical_imputation",ArbitraryNumberImputer(arbitrary_number=-1,variables=["age","fare"])),
    ("categorical_imputation",CategoricalImputer(variables=["embarked"])),
    ("categorical_encoder",OrdinalEncoder(encoding_method="ordered",variables=["sex","embarked"])),
    ("rf",RandomForestClassifier(random_state=0))
])

In [9]:
#Applying the pipeline to the training set

titanic_data_pipe.fit(X_train,y_train)
pred_X_train = titanic_data_pipe.predict(X_train)
pred_X_test = titanic_data_pipe.predict(X_test)

In [10]:
#Evaluating the performance accuracy of the predictions made on the test set 

from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

print(confusion_matrix(y_test,pred_X_test))
print(classification_report(y_test,pred_X_test))
print(accuracy_score(y_test,pred_X_test))

[[90 15]
 [19 55]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

0.8100558659217877


In [11]:
# With data preparation we were able to achieve an accuracy of 81% on predicting titanic passenger survival

# Regression Project

In [12]:
#Loading the diamonds dataset

diamond_data = sns.load_dataset("diamonds")

diamond_data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [13]:
#Checking the data type for each column

diamond_data.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

In [14]:
#Checking to see if our dataset contains any missing values 

diamond_data.isnull().mean()

carat      0.0
cut        0.0
color      0.0
clarity    0.0
depth      0.0
table      0.0
price      0.0
x          0.0
y          0.0
z          0.0
dtype: float64

In [15]:
#Looks lke there are no missing values

In [16]:
#Splitting the dataset into training and test set

X_train,X_test,y_train,y_test = train_test_split(diamond_data.drop("price",axis=1),diamond_data["price"],test_size=0.2,random_state=42)

X_train.shape,X_test.shape

((43152, 9), (10788, 9))

In [17]:
#Creating a pipeline to convert the categorical columns to numeric using one-hot encoding and to run the RandomForestRegressor to
#predict the prices of diamonds

diamond_data_pipe = Pipeline([
    ("categorical_encoder",OrdinalEncoder(encoding_method = "ordered",variables=["cut","color","clarity"])),
    ("rf",RandomForestRegressor(random_state=42))
])

In [18]:
diamond_data_pipe.fit(X_train,y_train)

pred_X_train = diamond_data_pipe.predict(X_train)
pred_X_test = diamond_data_pipe.predict(X_test)

In [19]:
#Testing to see how good the prediction are

from sklearn import metrics

print("Mean Absolute Error: ",metrics.mean_absolute_error(y_test,pred_X_test))
print("Mean Squared Error: ",metrics.mean_squared_error(y_test,pred_X_test))
print("Root Mean Squared Error: ",np.sqrt(metrics.mean_squared_error(y_test,pred_X_test)))

Mean Absolute Error:  271.5354506226789
Mean Squared Error:  308115.6650668038
Root Mean Squared Error:  555.0816742307422


In [20]:
# Finding the mean diamond price

diamond_data["price"].mean()

3932.799721913237

In [21]:
#The output shows the mean absolute error is 271.53 which is less than 10 percent of the mean price value which is 3,932 this shows
#that our predictions were very close to the actual price values